In [0]:
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

batch_size = 128
num_epochs = 50
learning_rate = 1e-3

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [2]:
train_set = DataLoader(datasets.CIFAR10('cifar',train=True, transform=transforms.Compose(
    [
     transforms.RandomHorizontalFlip(),
    #  transforms.RandomVerticalFlip(),
     transforms.RandomResizedCrop(size=(28,28), scale = (0.8, 1.0)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ]
), download = True), batch_size = batch_size, shuffle = True)

val_set = DataLoader(datasets.CIFAR10('cifar',train=False, transform=transforms.Compose(
    [
    #  transforms.RandomHorizontalFlip(),
    #  transforms.RandomVerticalFlip(),
    #  transforms.RandomResizedCrop(size=(28,28), scale = (0.7, 1.0)),
     transforms.Resize((28,28)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
    ]
), download = True), batch_size = batch_size, shuffle = False)

print('Dataset is prepared')

Files already downloaded and verified
Files already downloaded and verified
Dataset is prepared


In [3]:


resnet = models.resnet34(pretrained=True)
model = nn.Sequential(
    *list(resnet.children())[0:-1],
    nn.Flatten(),
    nn.ReLU(),
    nn.Linear(512,128),
    nn.ReLU(),
    nn.Linear(128,32),
    nn.ReLU(),
    nn.Linear(32,10)
)
print(device)
model = model.to(device)

model.eval()
test_x = torch.rand((1,3,28,28)).to(device)
test_y = model(test_x).to(device)
print(test_y.shape)

crit = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr = learning_rate, weight_decay=0.01)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience = 10, verbose = True)

print('Model and utils are prepared')

cuda
torch.Size([1, 10])
Model and utils are prepared


In [4]:
model.train()
for epoch in range(num_epochs):
    for idx, (x, y) in enumerate(train_set):
        x, y = x.to(device), y.to(device)
        prob = model(x)
        loss = crit(prob, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        period = 100
        if idx % period == period-1:
            print('Epoch: {}, IDX: {}, loss: {}'.format(epoch, idx+1, loss.item()))
    scheduler.step(loss)

Epoch: 0, IDX: 100, loss: 1.401452660560608
Epoch: 0, IDX: 200, loss: 1.3422857522964478
Epoch: 0, IDX: 300, loss: 0.9995803833007812
Epoch: 1, IDX: 100, loss: 1.3104020357131958
Epoch: 1, IDX: 200, loss: 1.057896614074707
Epoch: 1, IDX: 300, loss: 1.1771559715270996
Epoch: 2, IDX: 100, loss: 1.0458505153656006
Epoch: 2, IDX: 200, loss: 1.0735793113708496
Epoch: 2, IDX: 300, loss: 1.1337192058563232
Epoch: 3, IDX: 100, loss: 1.0588536262512207
Epoch: 3, IDX: 200, loss: 1.0220519304275513
Epoch: 3, IDX: 300, loss: 1.0049775838851929
Epoch: 4, IDX: 100, loss: 0.9719308018684387
Epoch: 4, IDX: 200, loss: 0.7850475311279297
Epoch: 4, IDX: 300, loss: 0.9888703227043152
Epoch: 5, IDX: 100, loss: 0.9346638917922974
Epoch: 5, IDX: 200, loss: 0.9159506559371948
Epoch: 5, IDX: 300, loss: 0.7396653294563293
Epoch: 6, IDX: 100, loss: 0.8716407418251038
Epoch: 6, IDX: 200, loss: 0.9627891182899475
Epoch: 6, IDX: 300, loss: 1.1464289426803589
Epoch: 7, IDX: 100, loss: 0.9544209837913513
Epoch: 7, ID

In [5]:
from google.colab import drive
drive.mount('/content/drive')
drive_folder = '/content/drive/My Drive/'
model_name = 'cifar10_resnet.pt'
torch.save(model.state_dict(),drive_folder+model_name)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
model.eval()
total_correct = 0
for x, y in val_set:
    x, y = x.to(device), y.to(device)
    prob = model(x)
    prid = torch.argmax(prob, dim = 1)
    total_correct += torch.sum(prid.eq(y)).float().item()

total = len(val_set.dataset)
acc = total_correct/total
print('total_correct: {}, total: {}, acc: {}'.format(total_correct, total, acc))

total_correct: 8076.0, total: 10000, acc: 0.8076
